In [2]:
from datasets import load_dataset
ted=load_dataset('csv',data_files={'train':'/home/nxingyu/data/ted_talks_processed.train.csv',
                                         'dev':'/home/nxingyu/data/ted_talks_processed.dev1.csv',
                                         'test':'/home/nxingyu/data/ted_talks_processed.test.csv'})

Using custom data configuration default
/home/nxingyu/miniconda3/envs/NLP/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607369981906/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


FileNotFoundError: [Errno 2] No such file or directory: '/home/nxingyu/data/ted_talks_processed.dev1.csv'

In [3]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

[INFO|file_utils.py:1214] 2020-12-16 13:44:13,850 >> https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /home/nxingyu/.cache/huggingface/transformers/tmpgd0qlfht


[INFO|file_utils.py:1218] 2020-12-16 13:44:14,134 >> storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at /home/nxingyu/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
[INFO|file_utils.py:1221] 2020-12-16 13:44:14,135 >> creating metadata file for /home/nxingyu/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
[INFO|tokenization_utils_base.py:1768] 2020-12-16 13:44:14,136 >> loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/nxingyu/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
[INFO|tokenization_utils_base.py:1768] 2020-12-16 13:44:14,136 >> loading file https://huggin

In [192]:
import regex as re
import numpy as np

tags=sorted(list('.?!,;:-—…'))
tag2id = {tag: id+1 for id, tag in enumerate(tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

def text2masks(n):
    def text2masks(text):
        '''Converts single paragraph of text into a list of words and corresponding punctuation based on the degree requested.'''
        if n==0: 
            refilter="(?<=[.?!,;:\-—… ])(?=[^.?!,;:\-—… ])|$" 
        else:
            refilter="[.?!,;:\-—…]{1,%d}(?= *[^.?!,;:\-—…]+|$)|(?<=[^.?!,;:\-—…]) +(?=[^.?!,;:\-—…])"%(n)
        word=re.split(refilter,text, flags=re.V1)
        punct=re.findall(refilter,text, flags=re.V1)
        wordlist,punctlist=([] for _ in range(2))
        for i in zip(word,punct+['']):
            w,p=i[0].strip(),i[1].strip()
            if w!='':
                wordlist.append(re.sub(r'[.?!,;:\-—… ]','',w))
                punctlist.append(0 if not w[-1] in '.?!,;:-—…' else tag2id[w[-1]])
            if p!='':
                wordlist.append(p)
                punctlist.append(0)
        return(wordlist,punctlist)
    return text2masks
assert(text2masks(0)('"Hello!!')==(['"Hello'], [1]))
assert(text2masks(1)('"Hello!!')==(['"Hello', '!'], [1, 0]))
assert(text2masks(0)('"Hello!!, I am human.')==(['"Hello','I','am','human'], [2,0,0,4]))
assert(text2masks(2)('"Hello!!, I am human.')==(['"Hello', '!,','I','am','human','.'], [1,0,0,0,0,0]))
def chunk_examples_with_degree(n):
    '''Ensure batched=True if using dataset.map or ensure the examples are wrapped in lists.'''
    def chunk_examples(examples):
        output={}
        output['texts']=[]
        output['tags']=[]
        for sentence in examples['transcript']:
            text,tag=text2masks(n)(sentence)
            output['texts'].append(text)
            output['tags'].append([0]+tag if text[0]!='' else tag) # [0]+tag so that in all case, the first tag refers to [CLS]
        return output
    return chunk_examples
assert(chunk_examples_with_degree(0)({'transcript':['Hello!Bye…']})=={'texts': [['Hello', 'Bye']], 'tags': [[0, 1, 9]]})

def encode_tags(encodings, docs, stride=15):
    encoded_labels = []
    doc_id=0
    label_offset=1
#     print(encodings.keys())
    for doc_offset,current_doc_id in zip(encodings.offset_mapping,encodings['overflow_to_sample_mapping']):
#         print(doc_id, end=' ')
        if current_doc_id>doc_id:
            doc_id+=1
            label_offset=0
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        doc_enc_labels[0]=docs[doc_id][label_offset-1] # Set leading punctuation class
#         print([id2tag[t] if t>0 else '' for t in docs[doc_id][label_offset:label_offset+len(doc_offset)]])
        arr_offset = np.array(doc_offset)
        arr_mask = (arr_offset[:,0] == 0) & (arr_offset[:,1] != 0) # Gives the labels that should be assigned punctuation
        doc_enc_labels[arr_mask] = docs[doc_id][label_offset:label_offset+sum(arr_mask)]
        encoded_labels.append(doc_enc_labels)            
        label_offset+=sum(arr_mask[:stride+1])
        print('.', end='')
    return encoded_labels

def process_dataset(dataset,split):
    data=dataset[split].map(chunk_examples_with_degree(0), batched=True, batch_size=32,remove_columns=dataset[split].column_names)
    encodings=tokenizer(data['texts'], is_split_into_words=True, return_offsets_mapping=True,
              return_overflowing_tokens=True, padding=True, truncation=True, max_length=32, stride=15,)
    
    labels=encode_tags(encodings, data['tags'])
    encodings.pop("offset_mapping")
    encodings.pop("overflow_to_sample_mapping")
#     train_dataset = tf.data.Dataset.from_tensor_slices((
#         dict(train_encodings),
#         train_labels
#     ))
    return torch.utils.data.TensorDataset(torch.tensor(encodings['input_ids'],dtype=torch.long),
        torch.tensor(encodings['attention_mask'],dtype=torch.long),
        torch.tensor(labels,dtype=torch.long))
#     return {'input_ids':torch.tensor(encodings['input_ids'],dtype=torch.long),
#             'attention_mask':torch.tensor(encodings['attention_mask'],dtype=torch.int32),
#             'target_punctuation':torch.tensor(labels,dtype=torch.int32)}
# train_dataset=dataset2tf(ted,'train')
# dev_dataset=dataset2tf(ted,'dev')
# test_dataset=dataset2tf(ted,'test')

In [94]:
dev_encodings=process_dataset(ted,'dev')

Loading cached processed dataset at /home/nxingyu/.cache/huggingface/datasets/csv/default-f6b55550f42cdbb1/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2/cache-643dd6445e962207.arrow


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [193]:
train_dataset=process_dataset(ted,'train')
dev_dataset=process_dataset(ted,'dev')
test_dataset=process_dataset(ted,'test')
for name,dataset in {'train':train_dataset, 'dev':dev_dataset, 'test':test_dataset}.items():
    torch.save(dataset, '/home/nxingyu/data/ted-'+name+'.pt')

Loading cached processed dataset at /home/nxingyu/.cache/huggingface/datasets/csv/default-f6b55550f42cdbb1/0.0.0/2960f95a26e85d40ca41a230ac88787f715ee3003edaacb8b1f0891e9f04dda2/cache-99664727baba489e.arrow


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

...

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

.................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [132]:
for name,dataset in {'train':train_dataset, 'dev':dev_dataset, 'test':test_dataset}.items():
    torch.save(dataset, '/home/nxingyu/data/ted-'+name+'.pt')

In [134]:
for name,dataset in {'train':train_dataset, 'dev':dev_dataset, 'test':test_dataset}.items():
    dataset=torch.load('/home/nxingyu/data/ted-'+name+'.pt')

In [181]:
# {'input_ids':torch.tensor(dev_encodings[0],dtype=torch.long),
# 'attention_mask':torch.tensor(dev_encodings[1],dtype=torch.int32),
# 'target_punctuation':torch.tensor(dev_encodings[2],dtype=torch.int32)}
for i in next(iter(dev_dataloader)):
    i=i.to('cpu')

In [174]:
for i in iter(dev_dataloader):
    print(i)

[tensor([[  101,  1045,  2215,  ...,  2066,  2035,   102],
        [  101,  2784,  1998,  ...,  2157,  2045,   102],
        [  101,  2122,  3471,  ...,  2619,  2066,   102],
        ...,
        [  101,  1005,  1055,  ...,  1997,  1996,   102],
        [  101,  1997,  2019,  ...,  1997,  6605,   102],
        [  101, 19388,  2024,  ...,  1996,  4471,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100],
        ...,
        [   0, -100, -100,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100]], dtype=torch.int32)]
[tensor([[  101,  1037,  3291,  ...,  2066, 11713,  

[tensor([[  101,  1996,  2132,  ...,  2987,  1005,   102],
        [  101,  2066,  2396,  ...,  2021,  6600,   102],
        [  101,  1056,  2191,  ...,  1037, 26917,   102],
        ...,
        [  101,  1037,  4861,  ..., 13675, 12322,   102],
        [  101,  2429,  2000,  ...,  3057, 13675,   102],
        [  101,  2015,  3191,  ...,  2001, 20547,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0, -100, -100],
        [   0,    8,    0,  ...,    8,    0, -100],
        [   0, -100,    0,  ...,    0,    0, -100],
        ...,
        [   0,    0,    0,  ...,    0, -100, -100],
        [   0,    2,    0,  ...,    0,    0, -100],
        [   0, -100,    0,  ...,    0,    0, -100]], dtype=torch.int32)]
[tensor([[  101, 12322,  2015,  ...,  1045,  2409,  

[tensor([[  101,  2488,  1999,  ...,  3473,  2039,   102],
        [  101, 11793,  2122,  ...,  2831,  2000,   102],
        [  101,  1999,  1996,  ...,  4223,  1998,   102],
        ...,
        [  101,  5188,  7590,  ...,  2770,  2026,   102],
        [  101,  2057,  2681,  ...,  2172,  1998,   102],
        [  101,  2391,  2003,  ...,  3084,  2068,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    2,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    4, -100],
        ...,
        [   0,    0,    0,  ...,    0,    4, -100],
        [   0,    4,    0,  ...,    0,    2, -100],
        [   4,    0,    0,  ...,    0,    0, -100]], dtype=torch.int32)]
[tensor([[  101,  2027,  2342,  ...,  5278,  2023,  

[tensor([[ 101, 2020, 2183,  ..., 4158, 2000,  102],
        [ 101, 1040, 2031,  ..., 5022, 2040,  102],
        [ 101, 2033, 2045,  ..., 2560, 3497,  102],
        ...,
        [ 101, 3697, 2000,  ..., 2000, 8116,  102],
        [ 101, 1997, 3941,  ..., 1037, 7997,  102],
        [ 101, 2740, 2729,  ..., 4425, 2003,  102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0,    0, -100],
        [   4, -100,    0,  ...,    0,    0, -100],
        [   0,    0,    5,  ...,    0,    0, -100],
        ...,
        [   0,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100]], dtype=torch.int32)]
[tensor([[  101,  2007,  1037,  ...,  1037,  5943,   102],
        [  101,  9462,  2011,

[tensor([[ 101, 1998, 2087,  ..., 2022, 2045,  102],
        [ 101, 1005, 1055,  ..., 2092, 2058,  102],
        [ 101, 1045, 2228,  ..., 2008, 2041,  102],
        ...,
        [ 101, 2008, 2097,  ..., 1998, 2027,  102],
        [ 101, 2003, 2008,  ..., 2156, 1996,  102],
        [ 101, 2024, 2183,  ..., 1997, 2008,  102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    2,    0,  ...,    0,    0, -100],
        [   0, -100, -100,  ...,    0,    0, -100],
        [   0,    7,    0,  ...,    0,    0, -100],
        ...,
        [   0,    0,    0,  ...,    2,    0, -100],
        [   0,    2,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100]], dtype=torch.int32)]
[tensor([[  101,  7375,  1045,  ...,  1999, 15338,   102],
        [  101,  2005,  2017,

...

[tensor([[  101,  1056,  2228,  ...,  2009,  1998,   102],
        [  101,  2057,  2342,  ...,  2009,  2092,   102],
        [  101,  2748,  1045,  ...,  2657,  2004,   102],
        ...,
        [  101,  2019, 27352,  ...,  2026, 27352,   102],
        [  101,  1037,  2047,  ...,  2862,  2008,   102],
        [  101,  2895,  2018,  ...,  2022,  1999,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0, -100,    0,  ...,    0,    4, -100],
        [   4,    0,    0,  ...,    0,    0, -100],
        [   4,    0,    2,  ...,    0,    2, -100],
        ...,
        [   0,    0,    0,  ...,    2,    0, -100],
        [   2,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    2, -100]], dtype=torch.int32)]
[tensor([[  101,  2443,  1996,  ...,  2308,  1998,  

[tensor([[ 101, 2132, 1997,  ..., 1999, 7041,  102],
        [ 101, 2216, 2093,  ..., 3564, 2012,  102],
        [ 101, 2057, 2253,  ..., 1037, 2773,  102],
        ...,
        [ 101, 3112, 2466,  ..., 4952, 2000,  102],
        [ 101, 1005, 1056,  ..., 2058, 1996,  102],
        [ 101, 2111, 2027,  ..., 2924, 4952,  102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0,    0, -100],
        [   0,    4,    0,  ...,    0,    0, -100],
        [   0,    4,    0,  ...,    0,    0, -100],
        ...,
        [   0,    4,    0,  ...,    0,    0, -100],
        [   0, -100, -100,  ...,    0,    0, -100],
        [   0,    0,    4,  ...,    0,    2, -100]], dtype=torch.int32)]
[tensor([[  101,  2088,  2123,  ...,  2088,  1045,   102],
        [  101,  2000,  1996,

[tensor([[  101,  2000,  2228,  ...,  3613,  2000,   102],
        [  101,  2812,  2045,  ...,  2776,  4983,   102],
        [  101,  3857,  3001,  ...,  1005,  1055,   102],
        ...,
        [  101,  1005,  1056,  ...,  3177,  2894,   102],
        [  101,  2009,  1005,  ...,  2008,  2001,   102],
        [  101,  2157,  2061,  ...,  4816, 13782,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0,    0, -100],
        [   7,    0,    2,  ...,    0,    2, -100],
        [   0,    0,    0,  ..., -100, -100, -100],
        ...,
        [   0, -100, -100,  ...,    0,    0, -100],
        [   4,    0, -100,  ...,    0,    0, -100],
        [   0,    4,    7,  ...,    2,    0, -100]], dtype=torch.int32)]
[tensor([[  101,  2053, 25670,  ...,  2323,  2228,  

[tensor([[  101,  2562,  6230,  ...,  2071, 11025,   102],
        [  101,  2005,  4960,  ...,  2017, 24095,   102],
        [  101,  6599,  2041,  ...,  2090,  2047,   102],
        ...,
        [  101,  2128,  2025,  ...,  5957,  2001,   102],
        [  101, 22630, 12350,  ...,  2021,  2085,   102],
        [  101,  2467,  2081,  ...,  1996, 11485,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    0,  ...,    0,    0, -100],
        [   2,    0,    0,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    0, -100],
        ...,
        [   0, -100,    0,  ...,    0,    0, -100],
        [   0,    0,    2,  ...,    4,    0, -100],
        [   0,    0,    0,  ...,    2,    0, -100]], dtype=torch.int32)]
[tensor([[  101,  2045,  1005,  ...,  2027,  2024,  

[tensor([[  101,  2291,  2008,  ...,  1996,  2126,   102],
        [  101,  2041,  1998,  ..., 13767,  2015,   102],
        [  101,  4092,  1997,  ...,  2545,  3389,   102],
        ...,
        [  101, 18706,  2215,  ...,  2067, 29020,   102],
        [  101,  4895,  9148,  ...,  2000,  9017,   102],
        [  101,  2000,  4607,  ...,  9913,  2633,   102]]), tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32), tensor([[   0,    0,    2,  ...,    0,    0, -100],
        [   0,    0,    0,  ..., -100, -100, -100],
        [   0,    2,    0,  ..., -100,    3, -100],
        ...,
        [   0,    0,    0,  ...,    0, -100, -100],
        [   0,    0, -100,  ...,    0,    0, -100],
        [   0,    0,    0,  ...,    0,    4, -100]], dtype=torch.int32)]
[tensor([[  101,  1996, 17002,  ...,  2222,  2156,  

In [150]:
train_dataset[:,0][0].size()

torch.Size([414226])

In [183]:
import transformers
bert=transformers.BertModel.from_pretrained('distilbert-base-uncased')

[INFO|configuration_utils.py:411] 2020-12-16 17:21:28,582 >> loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /home/nxingyu/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361
[INFO|configuration_utils.py:447] 2020-12-16 17:21:28,583 >> Model config BertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "attention_probs_dropout_prob": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dim": 3072,
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "n_heads": 12,
  "n_layers": 6,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,

In [191]:
for i in dev_dataloader:
    a,b,c=i
    o1=bert(a,attention_mask=b)[0]
    print(o1)
    break

tensor([[[ 0.5464,  0.1706, -0.8875,  ...,  0.1214, -0.8452,  0.7705],
         [-0.7301,  0.0582,  1.6603,  ..., -0.2291, -0.9668,  1.3627],
         [-0.0893, -0.3870,  1.5180,  ...,  0.1244, -0.3943, -0.0938],
         ...,
         [ 0.4649,  0.3001,  1.0311,  ...,  0.5345,  0.0387,  0.9275],
         [-1.2800,  0.7345,  1.8155,  ...,  1.3324, -0.4411,  0.9953],
         [-0.1813, -0.8365,  0.2855,  ...,  0.2597, -0.5997,  0.5862]],

        [[ 0.6425,  0.3303, -0.9751,  ..., -0.1276, -0.6688,  0.8087],
         [-0.8156,  0.5898,  1.5734,  ...,  0.9461, -1.1702,  0.4226],
         [-0.0816,  0.2801, -0.0726,  ..., -0.2242, -0.2785,  1.2853],
         ...,
         [-0.3176,  1.3420,  1.2047,  ..., -1.4294, -1.1119,  0.4684],
         [ 0.0734,  1.5812,  2.0381,  ...,  0.9963,  0.1314,  0.7964],
         [-0.0749, -0.5730,  0.2179,  ..., -0.0284, -0.3980,  0.6043]],

        [[ 0.6698,  0.4314, -0.9165,  ..., -0.1959, -0.5605,  1.1532],
         [-0.2288,  0.3077,  1.2107,  ...,  0

In [171]:
for i in dev_dataloader:
    i=i.to(device)

AttributeError: 'list' object has no attribute 'to'

In [102]:
# import torch
dev_dataloader=torch.utils.data.DataLoader(dev_dataset,batch_size=32,num_workers=4)

In [165]:
for i in dev_dataloader:
    print(i[0])

tensor([[  101,  1045,  2215,  ...,  2066,  2035,   102],
        [  101,  2784,  1998,  ...,  2157,  2045,   102],
        [  101,  2122,  3471,  ...,  2619,  2066,   102],
        ...,
        [  101,  1005,  1055,  ...,  1997,  1996,   102],
        [  101,  1997,  2019,  ...,  1997,  6605,   102],
        [  101, 19388,  2024,  ...,  1996,  4471,   102]])
tensor([[  101,  1037,  3291,  ...,  2066, 11713,   102],
        [  101,  1998,  2059,  ...,  2054,  2064,   102],
        [  101,  1005,  1055,  ...,  1045,  2215,   102],
        ...,
        [  101,  2945,  1998,  ...,  2068,  2061,   102],
        [  101,  2599,  1998,  ...,  2000,  5454,   102],
        [  101, 11713,  1998,  ...,  2111,  2359,   102]])
tensor([[ 101, 2013, 5921,  ..., 2147, 2000,  102],
        [ 101, 2000, 3693,  ..., 2191, 1037,  102],
        [ 101, 6133, 1037,  ..., 2054, 2097,  102],
        ...,
        [ 101, 2061, 1045,  ..., 2651, 2256,  102],
        [ 101, 4187, 7692,  ..., 7574, 4896,  102],
   

tensor([[  101,  1997,  2607,  ...,  1037,  2126,   102],
        [  101,  2017,  2113,  ...,  2396,  2038,   102],
        [  101,  1996,  3274,  ...,  1998,  2061,   102],
        ...,
        [  101,  2409,  2033,  ...,  2025,  2183,   102],
        [  101,  3924,  2017,  ...,  1997, 13071,   102],
        [  101,  2000,  2022,  ...,  2972,  2406,   102]])
tensor([[ 101, 2105, 1045,  ..., 2131, 2000,  102],
        [ 101, 2000, 3751,  ..., 2038, 2000,  102],
        [ 101, 1037, 5576,  ..., 2482, 2021,  102],
        ...,
        [ 101, 1005, 1056,  ..., 2197, 3329,  102],
        [ 101, 2031, 1037,  ..., 1997, 1996,  102],
        [ 101, 1999, 1037,  ..., 5391, 2011,  102]])
tensor([[  101,  6502,  3791,  ...,  2215,  2000,   102],
        [  101,  2651,  1005,  ...,  2846,  2005,   102],
        [  101,  2994,  2306,  ...,  2057,  2794,   102],
        ...,
        [  101,  2043,  2027,  ...,  1998,  2057,   102],
        [  101,  1005,  2310,  ...,  2200,  5875,   102],
        [

tensor([[  101,  2488,  2125,  ...,  1005,  2222,   102],
        [  101,  2412,  8007,  ...,  2138,  2004,   102],
        [  101,  9120,  2023,  ...,  2000,  3275,   102],
        ...,
        [  101,  1005,  1055,  ...,  2087,  3937,   102],
        [  101,  2561, 18185,  ...,  2057,  2453,   102],
        [  101,  2504,  2061,  ...,  1045,  1005,   102]])
tensor([[  101,  5454,  2000,  ...,  4990,  2046,   102],
        [  101,  1049,  2025,  ...,  2000,  3857,   102],
        [  101,  1996,  2925,  ...,  2643, 23657,   102],
        ...,
        [  101,  9932,  6950,  ...,  2381,  2292,   102],
        [  101,  4895, 28139,  ...,  9867,  1998,   102],
        [  101,  1005,  1055,  ...,  2203,  2039,   102]])
tensor([[  101,  6114,  1998,  ..., 26108,  1996,   102],
        [  101,  1999,  1037,  ...,  2085,  6865,   102],
        [  101,  4138,  2024,  ...,  2187,  2079,   102],
        ...,
        [  101,  2085,  2009,  ...,  2622,  2000,   102],
        [  101,  3579,  2003,  

tensor([[  101,  2013,  1996,  ...,  2009, 16142,   102],
        [  101,  2000,  2191,  ...,  1055,  2242,   102],
        [  101,  2023,  3151,  ...,  2112,  1997,   102],
        ...,
        [  101,  2156,  2013,  ...,  2455,  2396,   102],
        [  101,  2045,  1005,  ...,  2027,  1005,   102],
        [  101,  2073,  2111,  ...,  7258,  1999,   102]])
tensor([[  101,  2128, 13071,  ...,  2682,  2061,   102],
        [  101,  2216, 23694,  ...,  2015,  1996,   102],
        [  101,  2054,  2057,  ...,  3698,  2005,   102],
        ...,
        [  101, 10796,  1998,  ...,  2081,  2068,   102],
        [  101,  2769,  2057,  ...,  2941,  2241,   102],
        [  101,  4895, 20192,  ...,  1037,  8117,   102]])
tensor([[  101,  2588,  2178,  ..., 10727,  3504,   102],
        [  101, 19779,  3480,  ...,  1999,  1037,   102],
        [  101,  2012,  1996,  ...,  3504,  2012,   102],
        ...,
        [  101,  1996,  2327,  ...,  2612,  2092,   102],
        [  101,  2202,  2122,  

tensor([[  101,  2008,  2138,  ...,  2157,  3182,   102],
        [  101,  2113,  2057,  ...,  2068,  1045,   102],
        [  101,  2000,  3828,  ...,  6162,  2256,   102],
        ...,
        [  101, 14175,  2098,  ...,  2014,  9504,   102],
        [  101,  2014,  3008,  ...,  7098,  1997,   102],
        [  101, 16391,  2038,  ...,  3020,  2495,   102]])
tensor([[  101,  1996,  2137,  ...,  1005,  1056,   102],
        [  101,  2021, 16391,  ...,  2003,  2013,   102],
        [  101,  3477,  1996,  ...,  2006,  2010,   102],
        ...,
        [  101,  2008,  2097,  ...,  2008,  7484,   102],
        [  101,  2129,  2057,  ...,  2006,  2000,   102],
        [  101,  5534,  2123,  ...,  2000,  4737,   102]])
tensor([[  101,  2256,  2493,  ...,  3791,  2000,   102],
        [  101,  2055,  3977,  ...,  2493,  2123,   102],
        [  101,  3233,  2012,  ...,  1997, 12655,   102],
        ...,
        [  101,  1005,  1056,  ...,  2007,  2149,   102],
        [  101,  3076,  2013,  

...

tensor([[  101,  2712,  2008,  ...,  2005,  1996,   102],
        [  101,  6727,  1996,  ...,  2589,  1037,   102],
        [  101,  2925,  1997,  ...,  9788, 19016,   102],
        ...,
        [  101,  6610,  2023,  ...,  5469,  1997,   102],
        [  101,  1998, 19035,  ...,   102,     0,     0],
        [  101,  2065,  2017,  ...,  2094,  1998,   102]])
tensor([[  101,  2017,  5454,  ...,  8491, 10250,   102],
        [  101,  2009,  1005,  ..., 21770, 10624,   102],
        [  101, 18909,  2174,  ...,  7685, 14615,   102],
        ...,
        [  101,  1045,  2018,  ...,  2134,  1005,   102],
        [  101,  2092,  5525,  ...,  2021,  1045,   102],
        [  101,  1056,  2031,  ...,  1045,  2973,   102]])
tensor([[  101, 10373,  2098,  ...,  2033,  2087,   102],
        [  101,  1998,  1045,  ...,  1996,  2051,   102],
        [  101,  1997,  1996,  ...,  2061,  2009,   102],
        ...,
        [  101,  1999,  2115,  ...,  2017,  1005,   102],
        [  101,  2008,  2065,  

tensor([[  101,  2003,  1037,  ...,  2023,  2077,   102],
        [  101,  1999,  2023,  ...,  2685,  2008,   102],
        [  101,  7244,  2008,  ...,  1996,  2825,   102],
        ...,
        [  101,  2607,  2057,  ..., 23900,  3117,   102],
        [  101,  2183,  2006,  ...,  2061,  2017,   102],
        [  101,  7368,  2009,  ...,  9946,  2061,   102]])
tensor([[  101,  4952,  2000,  ...,  2008,  6904,   102],
        [  101,  2008,  2017,  ...,  2006,  1998,   102],
        [  101, 23736, 12426,  ...,  2003,  2525,   102],
        ...,
        [  101,  2525,  1999,  ...,  1999,  1996,   102],
        [  101,  2085,  2057,  ...,  2023,  2256,   102],
        [  101,  2379,  2925,  ..., 12020,  1998,   102]])
tensor([[  101,  8045,  5679,  ...,  2045,  2003,   102],
        [  101,  1996,  2126,  ...,  4022,  2000,   102],
        [  101,  2062,  2023,  ...,  1045,  2052,   102],
        ...,
        [  101, 11698,  2088,  ...,  2129,  2106,   102],
        [  101,  3722,  2477,  

tensor([[ 101, 2489, 3259,  ..., 2515, 4025,  102],
        [ 101, 2052, 2360,  ..., 5138, 1996,  102],
        [ 101, 1037, 2210,  ..., 2024, 6135,  102],
        ...,
        [ 101, 1996, 4500,  ..., 2126, 2058,  102],
        [ 101, 4095, 1999,  ..., 2169, 1997,  102],
        [ 101, 1996, 4448,  ..., 2183, 2000,  102]])
tensor([[ 101, 2122, 3785,  ..., 2369, 2060,  102],
        [ 101, 2272, 2000,  ..., 7594, 2058,  102],
        [ 101, 2493, 2024,  ..., 2012, 2054,  102],
        ...,
        [ 101, 2437, 2009,  ..., 2003, 2045,  102],
        [ 101, 2009, 1005,  ..., 5970, 1998,  102],
        [ 101, 2428, 2498,  ..., 2028, 3160,  102]])
tensor([[  101,  1037,  4440,  ..., 10623,  3593,   102],
        [  101, 26264,  3231,  ...,  3044,  1998,   102],
        [  101, 12388,  2021,  ..., 14624,  1998,   102],
        ...,
        [  101,  4089,  4425,  ...,  2047,  3655,   102],
        [  101, 17463, 14122,  ...,  2000,  2022,   102],
        [  101,  2021,  2074,  ...,  2017,  2

tensor([[  101,  2075,  2302,  ...,  2021,  2852,   102],
        [  101,  2477,  2066,  ...,  2776, 26463,   102],
        [  101, 13962,  2001,  ...,  1998,  4384,   102],
        ...,
        [  101,  2020,  6086,  ...,  2055,  2182,   102],
        [  101,  3145,  4062,  ...,  1005,  1055,   102],
        [  101, 17341,  6622,  ...,  2491,  1998,   102]])
tensor([[  101,  3615,  2000,  ...,  7860,  1997,   102],
        [  101,  9740,  2655,  ..., 27892,  1999,   102],
        [  101,  2256,  2051,  ...,  3424, 26591,   102],
        ...,
        [  101,  4939,  2066,  ...,  2009,  2005,   102],
        [  101,  3302,  2943,  ...,  2092,  2023,   102],
        [  101,  2742,  2057,  ...,  3618,  3302,   102]])
tensor([[ 101, 6433, 2190,  ..., 2057, 2064,  102],
        [ 101, 2943, 6459,  ..., 5733, 2013,  102],
        [ 101, 2689, 2037,  ..., 4127, 1997,  102],
        ...,
        [ 101, 2031, 2643,  ..., 2734, 2000,  102],
        [ 101, 1037, 2200,  ..., 2296, 2785,  102],
   

tensor([[  101,  2001,  1996,  ...,  2563,  2013,   102],
        [  101,  2009,  2347,  ..., 24497,  9349,   102],
        [  101,  5087,  1037,  ..., 11218,  2054,   102],
        ...,
        [  101,  2122,  7590,  ...,  1997,  1996,   102],
        [  101,  3652,  7073,  ...,  2027, 10107,   102],
        [  101,  4111,  2088,  ...,  7955,  1998,   102]])
tensor([[  101,  2061,  2008,  ...,  2000,  2709,   102],
        [  101,  1045,  3246,  ...,  2310,  2042,   102],
        [  101,  2000, 13600,  ...,  2613, 12857,   102],
        ...,
        [  101,  2204,  2154,  ...,  3888,  2879,   102],
        [  101,  1996,  2087,  ...,  1998,  4687,   102],
        [  101,  1045,  3473,  ...,  3191,  2055,   102]])
tensor([[  101,  2054,  2001,  ..., 19379, 27773,   102],
        [  101,  1996,  6429,  ...,  1045,  2288,   102],
        [  101,  5603,  1998,  ...,  2062, 26398,   102],
        ...,
        [  101,  2273,  3262,  ...,  1002,  1015,   102],
        [  101,  6945,  2029,  

In [13]:
train_encodings['input_ids'][-1],train_encodings['attention_mask'][-1]

([101,
  2342,
  2000,
  5157,
  17842,
  2005,
  2256,
  9896,
  2594,
  2058,
  19980,
  2015,
  1996,
  3690,
  1997,
  6397,
  4752,
  1999,
  2502,
  2951,
  2442,
  2203,
  4067,
  2017,
  2200,
  2172,
  102,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0])